<a href="https://colab.research.google.com/github/franklinokech/Geographic_distance_calculator/blob/master/Find_Distance_Between_Farms_and_CCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import key libraries
import pandas as pd
import numpy as np
import sklearn.neighbors

In [3]:
# Import Libraries for connecting to Google drive
!pip install  -q gspread

In [4]:
# Authenticate to Google drive and get the required dataset
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [5]:
# Import SSOT
# Get the google sheet file for jaribuni
sh = gc.open('SSOT Shamba Person_MASTER')
# Select Spreadsheet
# By title
worksheet = sh.worksheet('Shamba')
data = worksheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)

# Preview dataframe
df.head()

,kcode,region_id,polygon_id,location_id,sublocation_id,zone_id,homestead_location,fa_assigned_id,fo_assigned_id,fm_assigned_id,veritas_id,latitude,longitude,geopoint,farmer_official_name,common_name,availability_id,official_type_id,farmer_id,gender,contact_method_id,farmer_phone_number,hist_farmer_id,hist_farmer_name,landowner_id,cosignee_id,caretaker_name,caretaker_phone_number,tree_species_id,suitability,suitability_melia,suitability_eucalyptus,planting_year_id,planting_season_id,area_m2,shamba_equivqlence_size,tree_spacing,seedlings_planted,seedlings_planted_melia,seedlings_planted_eucalyptus,last_audit_date,last_audit_alive_trees,last_audit_survived_trees,shamba_status,ownership_status,contract_status,slo_url,contract_url,uuid,SI_index_euc,tree_status,harvest_operation,farmerfollowup_check
0,klf-2008-0001,kilifi,klf-2008-0001,Ngerenya,Ngerenya,zone-01-224,,Janet Sidi Nguma,Michael Mtengo Mundu,Alfred Mwatsuma Kiti,gase7,-3.524255766,39.87817724,point ( -3.524255766 39.87817724 ),Esther Kai Mwatunje,Mama Bandari,available_on_farm,national_id,id_2147628,Female,farmer,714232488,,,id_2147628,id_2138432,,,Eucalyptus,very suitable,,very suitable,2008,LR,2180,1.0,2.5x2.5,200,,200,12-06-17 19:04,167,167,active,,,https://drive.google.com/open?id=1EjXIzpjYAVqn...,https://drive.google.com/open?id=1PuYE-Xzjq3Zi...,7397d896-bdaf-11e9-8473-631210d8f882,SI14,,,
1,klf-2008-0002,kilifi,klf-2008-0002,Ngerenya,Ngerenya,zone-01-224,,Janet Sidi Nguma,Michael Mtengo Mundu,Alfred Mwatsuma Kiti,gase8,-3.528535743,39.87702892,point ( -3.528535743 39.87702892 ),Pahe Kaingu Fondo,Mama Rehema,available_on_farm,national_id,id_9773938,Female,farmer,702374834,,,id_9773938,id_27052844,,,Eucalyptus,very suitable,,very suitable,2008,LR,1530,1.0,2.5x2.5,200,,200,12-06-17 19:05,148,148,active,,,https://drive.google.com/open?id=15MONI3UOAGrc...,https://drive.google.com/open?id=17xNJM-wnSU2G...,7397d897-bdaf-11e9-8473-631210d8f882,SI14,,,
2,klf-2008-0003,kilifi,klf-2008-0003,Palakumi,Mariani/Vitsapuni,zone-01-208,,James Katana Karisa,Daidas Mwachiti,Tyson Sangura Wanyama,emgr206,-3.569655033,39.64606282,point ( -3.569655033 39.64606282 ),Kahindi Kazungu Kasingihi,Kahindi Kazungu,available_near_farm,national_id,id_9112244,Male,farmer,701694431,,,id_9112244,id_11870882,,,Eucalyptus,not suitable,,not suitable,2008,SR,190,1.0,2.5x2.5,200,,200,01-08-17 20:20,30,40,active,,,https://drive.google.com/open?id=15bQ5W2MvZgqd...,https://drive.google.com/open?id=1E8v57fDm5-bK...,7397d898-bdaf-11e9-8473-631210d8f882,SI10,,,
3,klf-2008-0004,kilifi,klf-2008-0004,Palakumi,Mariani/Vitsapuni,zone-01-128,kwa kiraga kaviha,Christine Nzai Kalisho,James Mwahunga Ngumo,Mathias Mwabaya Kalama,emgr467,-3.607335748,39.64600114,point ( -3.607335748 39.64600114 ),Kahaso Kiraga Kaviha,Mkaza Binzai,available_near_farm,national_id,id_31487899,Female,other_family_member,799474601,,,id_31487899,id_25205678,,,Eucalyptus,not suitable,,not suitable,2008,SR,1190,1.0,2.5x2.5,200,,200,05-12-17 22:00,74,158,active,,,https://drive.google.com/open?id=1IhlatEOLpM8g...,https://drive.google.com/open?id=1FjGwBZeeqOB_...,7397d899-bdaf-11e9-8473-631210d8f882,SI10,,,
4,klf-2008-0005,kilifi,klf-2008-0005,Ngerenya,Ngerenya,zone-01-224,,Janet Sidi Nguma,Michael Mtengo Mundu,Alfred Mwatsuma Kiti,gase4,-3.52362521,39.88113789,point ( -3.52362521 39.88113789 ),Joyce Dume Kadzenga,Mamake Maki,available_on_farm,national_id,id_13484471,Female,relative,,,,id_13484471,id_8459829,,,Eucalyptus,very suitable,,very suitable,2008,LR,1680,1.0,2.5x2.5,200,,200,13-06-17 9:06,86,107,active,,,https://drive.google.com/open?id=14dTHaTSJXTj-...,https://drive.google.com/open?id=1AwSCni4601Rp...,7397d89a-bdaf-11e9-8473-631210d8f882,SI14,,,


In [6]:
df = df[['kcode', 'latitude', 'longitude']]

df.head()

,kcode,latitude,longitude
0,klf-2008-0001,-3.524255766,39.87817724
1,klf-2008-0002,-3.528535743,39.87702892
2,klf-2008-0003,-3.569655033,39.64606282
3,klf-2008-0004,-3.607335748,39.64600114
4,klf-2008-0005,-3.52362521,39.88113789


In [7]:
# convert to correct datatypes
df.latitude = pd.to_numeric(df.latitude)
df.longitude = pd.to_numeric(df.longitude)

df.head()

,kcode,latitude,longitude
0,klf-2008-0001,-3.524256,39.878177
1,klf-2008-0002,-3.528536,39.877029
2,klf-2008-0003,-3.569655,39.646063
3,klf-2008-0004,-3.607336,39.646001
4,klf-2008-0005,-3.523625,39.881138


In [11]:
# Import SSOT
# Get the google sheet file for jaribuni
sh = gc.open('komaza gps cordinates')
# Select Spreadsheet
# By title
worksheet = sh.worksheet('data')
data = worksheet.get_all_values()
headers = data.pop(0)

df_2 = pd.DataFrame(data, columns=headers)

# Preview dataframe
df_2.head()

,name,point,lat,lon,routeXL
0,HQ,"-3.62363,39.84417",-3.62363,39.84417,"-3.62363,39.84417@HQ@"
1,CCA,"-3.6607243,39.826719",-3.6607243,39.826719,"-3.6607243,39.826719@CCA@"


In [12]:
df_2.drop(df_2.index[0], inplace=True)

In [13]:
df_2.head()

,name,point,lat,lon,routeXL
1,CCA,"-3.6607243,39.826719",-3.6607243,39.826719,"-3.6607243,39.826719@CCA@"


In [14]:
df_2.drop(['point', 'routeXL'], axis=1, inplace=True)

In [15]:
cols_to_rename = {
    'lat': 'latitude',
    'lon': 'longitude'
}

df_2.rename(columns=cols_to_rename, inplace=True)

In [16]:
df_2.head()

,name,latitude,longitude
1,CCA,-3.6607243,39.826719


In [17]:
df_2.latitude = pd.to_numeric(df_2.latitude)
df_2.longitude = pd.to_numeric(df_2.longitude)

In [18]:
# add columns with radians for latitude and longitude
df[['latitude_rad','longitude_rad']] = (
    np.radians(df.loc[:,['latitude','longitude']])
)
df_2[['latitude_rad','longitude_rad']] = (
    np.radians(df_2.loc[:,['latitude','longitude']])
)

In [19]:
df.head()

,kcode,latitude,longitude,latitude_rad,longitude_rad
0,klf-2008-0001,-3.524256,39.878177,-0.061510,0.696005
1,klf-2008-0002,-3.528536,39.877029,-0.061585,0.695985
2,klf-2008-0003,-3.569655,39.646063,-0.062302,0.691954
3,klf-2008-0004,-3.607336,39.646001,-0.062960,0.691953
4,klf-2008-0005,-3.523625,39.881138,-0.061499,0.696057


In [20]:
df_2.head()

,name,latitude,longitude,latitude_rad,longitude_rad
1,CCA,-3.660724,39.826719,-0.063892,0.695107


In [21]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (df[['latitude_rad','longitude_rad']],
     df_2[['latitude_rad','longitude_rad']])*6371
)
# Note that 6371 is the radius of the earth in km
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=df['kcode'], 
                 columns=df_2['name'])
)

In [22]:
df_dist_matrix.head()

name,CCA
kcode,
klf-2008-0001,16.213583
klf-2008-0002,15.723353
klf-2008-0003,22.460411
klf-2008-0004,20.914721
klf-2008-0005,16.397369


In [23]:
# Unpivot this dataframe from wide format to long format.
# When you unpivot, the data in the pivot table becomes a
# column named 'value'. Rename this column to 'miles' for clarity.
df_dist_long = (
    pd.melt(df_dist_matrix.reset_index(),id_vars='kcode')
)
df_dist_long = df_dist_long.rename(columns={'value':'km'})

In [24]:
df_dist_long.head()

,kcode,name,km
0,klf-2008-0001,CCA,16.213583
1,klf-2008-0002,CCA,15.723353
2,klf-2008-0003,CCA,22.460411
3,klf-2008-0004,CCA,20.914721
4,klf-2008-0005,CCA,16.397369


In [25]:
from google.colab import auth
from gspread_dataframe import get_as_dataframe, set_with_dataframe
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())



# import gsheet with the data
# Spreadsheet ID:1K5p6xrOy8t-v0vfYWDXm9WqIIgdaPtMNLoNjbXFhHk0

sh = gc.open('ROUTING DISTANCE ESTIMATE')

# Select Spreadsheet
# By title
worksheet = sh.worksheet('kcode_cca_distance')

In [26]:
# Append Dataframe to Sheet
set_with_dataframe(worksheet, df_dist_long)